In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import sqlite3
%matplotlib inline
from IPython.core.display import HTML
pd.options.display.max_columns = 50
import seaborn as sb
import datetime
from sqlalchemy import create_engine
import psycopg2
HTML("<style>.container { width:100% !important; }</style>")

In [3]:
#sql_conn_local = sqlite3.connect('flask_app/data/analysed_data.sql')
#sql_cursor_local = sql_conn.cursor()
conn = psycopg2.connect("dbname='grb_2016_03' host='forespellpostgis.cusejoju89w7.eu-west-1.rds.amazonaws.com' user='kasper' password='VosseM08'")
cur = conn.cursor()
engine_AWS = create_engine('postgresql://%s:%s@forespellpostgis.cusejoju89w7.eu-west-1.rds.amazonaws.com:5432/grb_2016_03' %('kasper', 'VosseM08'))

In [27]:
pd.read_sql('''SELECT * FROM namabayi.feedback LIMIT 10''', engine_AWS)

,feedback,name,session_id,time,user_id,sex,rank
0,no_like,Charlien,O1+UDVItalp5Bbj2I4YXPOFIQo3bg7oW,2016-09-28 20:26:40.927725,LyS8jqGkQaWTGm9lMrkcoffOjOH5lb7P,F,NaN
1,like,Cleo,gE6plbU54qjo/sqinvt/Xhs6rQxLqafF,2016-09-28 08:49:34.833191,LyS8jqGkQaWTGm9lMrkcoffOjOH5lb7P,F,1.0
2,no_like,Marion,aWy20Aii1BbwdMMqpCJ44m9D9Y9q8rsz,2016-09-28 20:28:51.022527,LyS8jqGkQaWTGm9lMrkcoffOjOH5lb7P,F,NaN
3,no_like,Yigit,ldqbaYN+JusTdCHBltQEKYiENU+2mEd3,2016-09-28 19:02:07.342758,O9e3mfef//f5dAiR2sqtxTY0QDybY4Gi,M,NaN
4,no_like,Timo,ldqbaYN+JusTdCHBltQEKYiENU+2mEd3,2016-09-28 19:02:10.330723,O9e3mfef//f5dAiR2sqtxTY0QDybY4Gi,M,NaN
5,no_like,Jacques,ldqbaYN+JusTdCHBltQEKYiENU+2mEd3,2016-09-28 19:02:10.922935,O9e3mfef//f5dAiR2sqtxTY0QDybY4Gi,M,NaN
6,like,Floris,3ckDw0fIezjXUqx62utv2f1gQt0tAPmU,2016-09-28 08:46:53.124234,LyS8jqGkQaWTGm9lMrkcoffOjOH5lb7P,M,2.0
7,like,Ensar,3ckDw0fIezjXUqx62utv2f1gQt0tAPmU,2016-09-28 08:46:44.988279,LyS8jqGkQaWTGm9lMrkcoffOjOH5lb7P,M,1.0
8,no_like,Karel,r252KYxBfQ35Kxz4sp/3rZA7ucgweioa,2016-09-28 18:57:51.366937,mTJqurZXfXDvatYTzo969AUzJ1CTdiGz,F,NaN
9,like,Caroline,r252KYxBfQ35Kxz4sp/3rZA7ucgweioa,2016-09-28 18:57:56.274951,mTJqurZXfXDvatYTzo969AUzJ1CTdiGz,F,NaN


## Create a new table

In [36]:
dict_to_store = {'best_feature':'origin_vl', 'user_id': 'LyS8jqGkQaWTGm9lMrkcoffOjOH5lb7P', 
                 'requested_sex':'M', 
            'value_left_child':4, 'value_right_child':5,
                'treshold' : 6, 'len_training_data':9}

In [37]:
tryout = pd.DataFrame([dict_to_store])
tryout.to_sql(name = 'decision_trees', schema = 'namabayi', con = engine_AWS, index=False, if_exists='replace')

In [4]:
# Give namabayi user again all the privilages
try:
    #cur.execute('''REVOKE CONNECT ON DATABASE grb_2016_03 FROM PUBLIC;''')
    #cur.execute('''GRANT CONNECT ON DATABASE grb_2016_03  TO namabayi_dev;''')

    cur.execute('''GRANT USAGE
                        ON SCHEMA namabayi
                        TO namabayi_dev; '''.replace('\n', ''))
    cur.execute('''GRANT ALL
                        ON ALL TABLES IN SCHEMA namabayi TO namabayi_dev ;'''.replace('\n', ''))    
    conn.commit()
except Exception as e:
    print(e)
    conn.rollback()

## Verify indexes on tables

In [39]:
query = '''select
    t.relname as table_name,
    i.relname as index_name,
    a.attname as column_name
from
    pg_class t,
    pg_class i,
    pg_index ix,
    pg_attribute a
where
    t.oid = ix.indrelid
    and i.oid = ix.indexrelid
    and a.attrelid = t.oid
    and a.attnum = ANY(ix.indkey)
    and t.relkind = 'r'
    and t.relname like %(table_name)s
order by
    t.relname,
    i.relname;'''

In [65]:
pd.read_sql(query, engine_AWS, params = {'table_name':'name_lookups'})

,table_name,index_name,column_name
0,name_lookups,name_lookups_name_1_idx,name_1
1,name_lookups,name_lookups_name_2_idx,name_2
2,name_lookups,name_lookups_region_idx,region
3,name_lookups,name_lookups_user_id_idx,user_id


In [64]:
try:
    cur.execute('''CREATE INDEX ON namabayi.name_lookups (name_1) ''')   
    cur.execute('''CREATE INDEX ON namabayi.name_lookups (name_2) ''')   
    cur.execute('''CREATE INDEX ON namabayi.name_lookups (region) ''')   
    cur.execute('''CREATE INDEX ON namabayi.name_lookups (user_id) ''')   
    conn.commit()
except Exception as e:
    print(e)
    conn.rollback()

## Trial and error code

In [18]:
suggestion = ['Karel', 'Sander', 'Kasper']
requested_sex = 'M'
user_ID = 'LyS8jqGkQaWTGm9lMrkcoffOjOH5lb7P'
session_ID = 'O1+UDVItalp5Bbj2I4YXPOFIQo3bg7oW'
to_store = pd.DataFrame({'name':suggestion, 'user_id':np.repeat(user_ID, len(suggestion)), 'session_id':np.repeat(session_ID, len(suggestion)),
             'requested_sex':np.repeat(requested_sex, len(suggestion))})
to_store.to_sql(name = 'awaiting_feedback', schema = 'namabayi', index=False, if_exists = 'append', con = engine_AWS)

In [63]:
tryout = pd.read_sql('''SELECT * FROM namabayi.name_lookups LIMIT 10''', engine_AWS)
tryout.head()

,name_1,name_2,region,session_id,sex_name_1,sex_name_2,time,user_id,from_landing_page
0,Caroline,,Belgie,c3WgKBWpbYWGfJyutOflae77822n2ACG,F,F,2016-09-17 20:17:20.899349,wbwCqGiw9/OnompOE6L6vCpIC93t53+U,False
1,,,Belgie,pprYNQidMNQt69aSzm2G6CQK/KIbUVg+,F,F,2016-09-17 20:19:28.923617,wbwCqGiw9/OnompOE6L6vCpIC93t53+U,False
2,Caroline,Karel,Belgie,pprYNQidMNQt69aSzm2G6CQK/KIbUVg+,F,F,2016-09-17 20:19:44.252927,wbwCqGiw9/OnompOE6L6vCpIC93t53+U,False
3,Caroline,Karel,Belgie,pprYNQidMNQt69aSzm2G6CQK/KIbUVg+,F,M,2016-09-17 20:19:49.808099,wbwCqGiw9/OnompOE6L6vCpIC93t53+U,False
4,Karel,Caroline,Belgie,ttCDD+WANqAHGuhwlRURWYX+mvEz0Afk,M,F,2016-09-18 17:08:17.346258,3gG4j0QstaS/DoYZfBTQMlCOfcoo7/0G,False


In [63]:
engine = create_engine('postgresql://%s:%s@forespellpostgis.cusejoju89w7.eu-west-1.rds.amazonaws.com:5432/grb_2016_03' %('namabayi_dev', 'namabayi_dev_40'))
conn = psycopg2.connect("dbname='grb_2016_03' host='forespellpostgis.cusejoju89w7.eu-west-1.rds.amazonaws.com' user='namabayi_dev' password='namabayi_dev_40'")
cur = conn.cursor()

In [20]:
#name_info = pd.read_sql('''SELECT * FROM voornamen_pivot WHERE region = 'Vlaanderen' AND sex = 'M'  LIMIT 10''', con=engine)
name_info = pd.read_sql('''SELECT name, minus18 FROM voornamen_pivot WHERE region = 'Vlaanderen' AND sex = 'M' ''', con=engine)

In [55]:
sessions = pd.read_sql('''SELECT * FROM sessions''', con=engine)
sessions.tail(1)

,ip_address,session_ID,time,user_ID,user_agent,window_height,window_width
2004,127.0.0.1,IwqnoDMNXDyGhUWX0KWy48eXKzfXKGDZ,2016-10-30 16:17:28.175706,+jMtz1zS2ES3/FYTLab2UuLR7aQ9xlEw,Mozilla/5.0 (iPhone; CPU iPhone OS 9_1 like Ma...,568,320


In [91]:
users = pd.read_sql('''SELECT * FROM registered_users ''', con=engine)
users

,email,password,registration_time,repeat_password,session_id,user_id
0,kasper.vanlombeek@gmail.com,pbkdf2:sha1:1000$Wyn71TU8$4b18e5fd9009a64a31f1...,2016-10-30 16:34:22.398307,pbkdf2:sha1:1000$od19ZZ5n$79b108fcd277a0db9040...,ONThR2UgJ11aJ3XtswH7W1mOyme1ZLAv,+jMtz1zS2ES3/FYTLab2UuLR7aQ9xlEw
1,kasper.vanlombeek@gmail.com,pbkdf2:sha1:1000$aWtGdBDn$8755722d67632dc976b0...,2016-10-30 16:34:56.247596,pbkdf2:sha1:1000$1lOm6auM$9cbe95493e45288ea211...,BxxjuR5+15m8/2WBri2XoiBVqAUWzjP8,+jMtz1zS2ES3/FYTLab2UuLR7aQ9xlEw
2,koen@gmail.com,pbkdf2:sha1:1000$lZylYOh0$b8dafca97d70ca6eb64f...,2016-10-30 16:44:28.464317,pbkdf2:sha1:1000$kAiAXu0r$4cf5b0b58e467a2b4226...,m+ZF6w+DL8PSoM4N2gBIpMxePBkOdbdC,+jMtz1zS2ES3/FYTLab2UuLR7aQ9xlEw
3,karel@gmail.com,pbkdf2:sha1:1000$uvCgmu9P$f5f064a056ed758f1c67...,2016-10-30 16:58:31.482368,pbkdf2:sha1:1000$xpETNte7$f447cfb7fcf873618e5a...,6ISLVAlqofMRRH/g8kH7LPtE8Er9P2zA,+jMtz1zS2ES3/FYTLab2UuLR7aQ9xlEw
4,louis@gmail.com,pbkdf2:sha1:1000$uu7LY5HI$c885b92a94e5a50a608b...,2016-10-30 17:11:58.455755,pbkdf2:sha1:1000$djhpb0nF$b0d4783664efb482bf68...,qvFPPjySXsAJoiyqPpSj/K9y0Q/f2sUu,+jMtz1zS2ES3/FYTLab2UuLR7aQ9xlEw
5,huguette@gmail.com,pbkdf2:sha1:1000$GyfpfJ6z$90c6880984f9454e8dfe...,2016-10-30 17:15:15.759667,pbkdf2:sha1:1000$v8OmfeUX$768173cacca5902d3e26...,+YKilKF3sF0DybkcLVHUnz+rkLpjjGSV,+jMtz1zS2ES3/FYTLab2UuLR7aQ9xlEw
6,sander@gmail.com,pbkdf2:sha1:1000$tUUTOTQ3$bd27b5a899611ccdfadc...,2016-10-30 17:38:26.510013,pbkdf2:sha1:1000$TtrYuCtb$e7ec614ec73ca7e9e970...,YMxDhXfoRFqSEZKNNwjLU8plMvqpmekF,+jMtz1zS2ES3/FYTLab2UuLR7aQ9xlEw
7,Kasper.vanlombeek@gmail.com,pbkdf2:sha1:1000$W9yPJiud$70c8c1574b50b96fc64b...,2016-10-30 17:28:58.997696,pbkdf2:sha1:1000$bRndO1Ty$413ed1d3683e3631606c...,r0N+vn719d9kVNnxcV46VEDuvOR2bOV4,5JVW0Me1JjkXvaTM7+0MB6V4NCjHK3Rn
8,Kasper.vanlombeek2@gmail.com,pbkdf2:sha1:1000$mtXtul3M$6267a1e9559ed7fd7064...,2016-10-30 17:29:33.784859,pbkdf2:sha1:1000$VoHkazjT$ffc5cf0524cfd41f7c10...,r0N+vn719d9kVNnxcV46VEDuvOR2bOV4,5JVW0Me1JjkXvaTM7+0MB6V4NCjHK3Rn
9,caroline@gmail.com,pbkdf2:sha1:1000$ls8Kfpzw$5c6079251544863b8c5c...,2016-10-30 17:32:28.751487,pbkdf2:sha1:1000$L8q5dCbu$88663adfdbf4ddb767b3...,cMf05P1CNI2Qza9BgbS9Z/QT2mYPrrkO,jVd2lPoEEP+psxLoqN434N6GzDPwdR7O


In [86]:
users['password'][0]

'pbkdf2:sha1:1000$uvCgmu9P$f5f064a056ed758f1c67c83fdc0b4aeec5fd88fb'

In [87]:
from werkzeug.security import generate_password_hash, check_password_hash
check_password_hash(password = 'karel', pwhash='pbkdf2:sha1:1000$uvCgmu9P$f5f064a056ed758f1c67c83fdc0b4aeec5fd88fb')

True

In [55]:
feedback = pd.read_sql('''SELECT * FROM feedback''', con=engine_AWS)
feedback.columns = [column.lower() for column in feedback.columns]
feedback.to_sql(con=engine_AWS, name='feedback',index=False, if_exists='replace')

In [68]:
pd.read_sql('''SELECT * FROM feedback''', con=engine_AWS).tail()

,feedback,name,session_id,time,user_id,sex
439,like,Emile,hMErCykJ4PQyxpKyKcVJZJEajQC5oJWt,2016-09-05 11:50:11.941045,wbwCqGiw9/OnompOE6L6vCpIC93t53+U,M
440,no_like,Richard,hMErCykJ4PQyxpKyKcVJZJEajQC5oJWt,2016-09-05 11:50:19.174217,wbwCqGiw9/OnompOE6L6vCpIC93t53+U,M
441,no_like,Sasha,hMErCykJ4PQyxpKyKcVJZJEajQC5oJWt,2016-09-05 11:50:54.944717,wbwCqGiw9/OnompOE6L6vCpIC93t53+U,M
442,no_like,Diego,hMErCykJ4PQyxpKyKcVJZJEajQC5oJWt,2016-09-05 11:51:12.248166,wbwCqGiw9/OnompOE6L6vCpIC93t53+U,M
443,no_like,Alexis,hMErCykJ4PQyxpKyKcVJZJEajQC5oJWt,2016-09-05 11:51:18.306556,wbwCqGiw9/OnompOE6L6vCpIC93t53+U,M


In [67]:
engine_AWS.execute('''DELETE FROM feedback WHERE time ='F' ''')